In [1]:
from matplotlib import pyplot as plt
import numpy as np
import sys 
import cv2

#creamos al clasificador
clasificador_rostro = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)


# Detección de esquinas para algoritmo Shi-tomasi
detector_esquina_params = dict( maxCorners = 200,
                       qualityLevel = 0.01,
                       minDistance = 10,
                       blockSize = 7 )

# Parametros para lucas Kanade (optical flow)
opticalFlow_params = dict( winSize  = (15,15),
                  maxLevel = 4, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# color aleatorio
color = np.random.randint(0,255,(100,3))

################################################################################################
# Capturar una primer imagen (Mirar defrente a la webcam para una buena captura)
ret, frame_previo = cap.read()
# muestra la captura
cv2.imshow('frame_previo', frame_previo)
cv2.waitKey(0)

################################################################################################
# Aqui detectamos al rostro usando el clasificador creado al inicio, de localizarlo lo cargamos en rostro_detectado

gris_frame_previo = cv2.cvtColor(frame_previo, cv2.COLOR_BGR2GRAY)
face = clasificador_rostro.detectMultiScale(gris_frame_previo, 1.2, 4)

if len(face) == 0:
    print ("No se detecto bien la cara, ponerla más defrente a la cámara")
    quit()

for (x,y,w,h) in face:
    rostro_detectado = frame_previo[y: y+h, x: x+w]

cv2.imshow('cara detectada', rostro_detectado )


################################################################################################
# Aqui detectamos las esquinas del rostro previamente detectado
# Dibujares circulos en la esquinas detectadas

rostro_detectado_gris = cv2.cvtColor(rostro_detectado,cv2.COLOR_BGR2GRAY)
esquinas_t = cv2.goodFeaturesToTrack(rostro_detectado_gris, mask = None, **detector_esquina_params)
esquinas = np.int0(esquinas_t)
esquinas_para_seguimiento = np.zeros(shape=(len(esquinas),1, 2) , dtype=np.float32 )

j=0
for i in esquinas:
    ix,iy = i.ravel()
    cv2.circle(rostro_detectado,(ix,iy),3,255,-1)
    cv2.circle(frame_previo,(x+ix,y+iy),3,255,-1)
    esquinas_para_seguimiento[j] = [[x+ix, y+iy]]
    j+=1    

cv2.imshow("viejo", frame_previo)
cv2.imshow("enfocado", rostro_detectado)


################################################################################################
#. Sobre las esquinas detectadas, realizamos el optical flow y dibujamos los puntos seguidos

p0 = esquinas_para_seguimiento

while(1):
    ret,frame = cap.read()
    frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Calculamos optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(gris_frame_previo, frame_gris, p0 , None, **opticalFlow_params)

    #Seleccionamos los mejores puntos
    mejores_nuevo_frame = p1[st==1]
    mejores_viejo_frame = p0[st==1]

    # Dibujar circulos
    for i,(nuevo,viejo) in enumerate(zip(mejores_nuevo_frame,mejores_viejo_frame)):
        a,b = nuevo.ravel()
        c,d = viejo.ravel()
        cv2.circle(frame,(a, b),5,color[i].tolist(),-1)
        if i == 99:
            break

            
    cv2.imshow('frame', frame)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Actualizando frame previo y puntos de las esquivas previos
    gris_frame_previo = frame_gris.copy()
    p0 = mejores_nuevo_frame.reshape(-1,1,2)
################################################################################################

cv2.destroyAllWindows()
cap.release()
cv2.waitKey(1)

-1

## Propuesta

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import math  
import sys 
import cv2
import time
    
#creamos al clasificador
clasificador_rostro = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Detección de esquinas para algoritmo Shi-tomasi
detector_esquina_params = dict( maxCorners = 200,
                       qualityLevel = 0.01,
                       minDistance = 10,
                       blockSize = 7 )

# Parametros para lucas Kanade (optical flow)
opticalFlow_params = dict( winSize  = (15,15),
                  maxLevel = 4, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# color aleatorio
color = np.random.randint(0,255,(100,3))

def mide_esparcimiento(p0, p1, porcent):
    cont=0
    for p in p0:
        dist = math.sqrt( (p1[0][0][0]- p[0][0])**2 + (p1[0][0][1]- p[0][1])**2)
        if(dist>100):
            cont=cont+1
            
    #res = np.subtract(p0, p1)
    total = p0.shape[0]
    pcnt = cont * 100 / total
    if(pcnt > porcent):
        print("Mi porcentaje: ",pcnt)
        print("Voy volver a medir");
        print("---------")
        return True
    else:
        return False
    
##########################    
#############Aqui empieza
###########################

cap = cv2.VideoCapture(0)

mide = False
continuar =False
mi_porcent = 50
while(1):
    
    ret,frame = cap.read()
    
    if(continuar):
        #print("Voy a continuar con el Optical")
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        #Calculamos optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(gris_frame_previo, frame_gris, p0 , None, **opticalFlow_params)
        #Seleccionamos los mejores puntos
        mejores_nuevo_frame = p1[st==1]
        mejores_viejo_frame = p0[st==1]

        # Dibujar circulos
        for i,(nuevo,viejo) in enumerate(zip(mejores_nuevo_frame,mejores_viejo_frame)):
            a,b = nuevo.ravel()
            c,d = viejo.ravel()
            cv2.circle(frame,(a, b),5,color[i].tolist(),-1)
            if i == 99:
                break
                
        # Mido el espacimiento de los puntos con Euclidiana        
        mide = mide_esparcimiento(p0,p1,mi_porcent)
        if(mide ==True):
            continuar = False

        # Actualizando frame previo y puntos de las esquivas previos
        gris_frame_previo = frame_gris.copy()
        p0 = mejores_nuevo_frame.reshape(-1,1,2)
        
    else:
        frame_previo = frame
        gris_frame_previo = cv2.cvtColor(frame_previo, cv2.COLOR_BGR2GRAY)
        face = clasificador_rostro.detectMultiScale(gris_frame_previo, 1.2, 4)

        if len(face) == 0:
            print ("No se detecto bien la cara, ponerla más defrente a la cámara")
            #quit()
        else:
            print("Rostro detectado")
            continuar = True
            for (x,y,w,h) in face:
                rostro_detectado = frame_previo[y: y+h, x: x+w]
                
            #dibujamos los puntos    
            rostro_detectado_gris = cv2.cvtColor(rostro_detectado,cv2.COLOR_BGR2GRAY)
            esquinas_t = cv2.goodFeaturesToTrack(rostro_detectado_gris, mask = None, **detector_esquina_params)
            esquinas = np.int0(esquinas_t)
            esquinas_para_seguimiento = np.zeros(shape=(len(esquinas),1, 2) , dtype=np.float32 )
            j=0
            for i in esquinas:
                ix,iy = i.ravel()
                cv2.circle(rostro_detectado,(ix,iy),3,255,-1)
                cv2.circle(frame_previo,(x+ix,y+iy),3,255,-1)
                esquinas_para_seguimiento[j] = [[x+ix, y+iy]]
                j+=1
            p0 = esquinas_para_seguimiento
            
    cv2.imshow('frame', frame)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
################################################################################################

cv2.destroyAllWindows()
cap.release()
cv2.waitKey(1)

Rostro detectado
Mi porcentaje:  59.0
Voy volver a medir
---------
Rostro detectado
